# Experiment A: Bound Adherence Under Nonstationary Conditions
*Question: does empirical average regret stay within the $\gamma$-regret bounds?*

### Experiment Design
Grid search over the streams (stationary, slow, abrupt, periodic), lambda levels, and two deletion ratio regimes.

Use a static comparator and calibrated learning rate schedule.
Set gamma per-theory at calibration and fix the total horizon (ie. $T=50000$).

### Primary Analysis
For each of the experimental cells, evaluate the final $R_{T}/T$ to see whether the guarantee was met.
Multiple seeds per cell in order to make some kind of causal analysis on the results.

### Success Criteria
95% of the cells meet their guarantees then I would call that a success. Which cells experience more successes than others? Can I make a heatmap that shows the regret results per cell?

## Import Statements and Versioning

This uses pretty standard library imports, but the torch requirement can stress the memory limits of a host.

In [ ]:
# set global seed
import random
import numpy as np
import torch

def set_seed(seed):
    """
    Set the random seed for reproducibility.
    
    Args:
        seed (int): The seed value to set for random number generation.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Config and Target

In [ ]:
RUN_ID = "experiment_a_test"
REGIME = "stationary"
REPLICATE = 1
T = 1000
SEED = 42
LOSS_NAME = "logistic"
MODEL_NAME = "memorypair"
LBFGS_MEM = 10

# Setup and Imports

In [ ]:
# Import required modules
import sys
import os

# Add code path for imports
os.chdir(os.path.dirname(os.getcwd()))

from config import Config
from runner import ExperimentRunner

# Test the d_max Fix

This section demonstrates that the d_max None value error has been fixed.

In [ ]:
# Test parameters that previously caused the error
test_params = {
    'gamma_bar': 1.0,
    'gamma_split': 0.3,
    'accountant': 'zcdp',
    'rho_total': 1.0,
    'delta_total': 1e-05,
    'max_events': 1000,  # Reduced for testing
    'delete_ratio': 0.05,
    'comparator': 'static',
    'd_max': None,  # This was causing the error
    'bootstrap_iters': 50,
    'quantile': 0.95,
    'dataset': 'synthetic',
    'seeds': 2
}

# Create configuration
set_seed(SEED)
cfg = Config.from_cli_args(**test_params)
print(f"Configuration created successfully!")
print(f"d_max value: {cfg.d_max} (should be inf, not None)")

# Create and test runner
runner = ExperimentRunner(cfg)
print(f"ExperimentRunner created successfully!")

# Run one seed to verify the fix
print(f"Running experiment for seed {SEED}...")
result = runner.run_one_seed(seed=SEED)
print(f"✅ Experiment completed successfully! No d_max comparison error.")
print(f"Result type: {type(result)}")

Configuration created successfully!
d_max value: inf (should be inf, not None)
ExperimentRunner created successfully!
Running experiment for seed 42...
[Bootstrap] Collecting 50 steps to estimate G, D, c, C...
[Bootstrap] Finalizing calibration...
[Calibrator] G_hat = 30.5823 (quantile 0.95)
[Calibrator] D_hat = 0.1771 (clamped by D_cap = 10.0)
[Calibrator] c_hat = 1.0000, C_hat = 1.0000
[MemoryPair] Calibration complete. N* = 327, transitioning to LEARNING phase.
[MemoryPair] Odometer will be finalized after warmup completes.
[SensCalib] Collecting 50 sensitivity samples...
[Warmup] Running 277 warmup inserts to reach N*=327...
[MemoryPair] Reached N* = 327 inserts. Ready to predict, transitioning to INTERLEAVING phase.
[Finalize] Finalizing odometer...
[ZCDPOdometer] Warning: Even m=1 exceeds constraints
[ZCDPOdometer] Required σ for m=1: 216.2493
[ZCDPOdometer] Regret bound: 495.3428 > γ=0.7000
[ZCDPOdometer] Joint optimization selected m=1, σ=216.2493
[ZCDPOdometer] Final regret bo

In [ ]:
# === Targets & accounting ===
gamma_bar:        [1.0]            # target average-regret level γ
gamma_split:      [0.3]            # % of γ reserved for unlearning/noise vs learner
accountant:       ["zcdp"]         # or add "legacy" to compare accountants in A/B runs
rho_total:        [1.0]            # used when accountant=zcdp
delta_total:      ["1e-5"]         # (for reporting / conversions)

# === Stream & schedule ===
stream_len:       [50000]          # total events T per run
drift_regime:     ["stationary", "slow", "abrupt", "periodic"]  # your generator should read this
delete_ratio:     [0.05, 0.2]      # deletes per insert (light/heavy)
# Optional: use a string-coded schedule if your runner supports it
# delete_schedule: ["poisson:lambda=0.005", "burst:k=5,B=500"]

# === Comparator & oracle (A focuses on static, add dynamic if desired) ===
comparator:       ["static"]       # add "dynamic" if you want both in one sweep
enable_oracle:    [false]          # true only if your dynamic comparator needs an oracle path
oracle_window_W:  [512]
oracle_steps:     [10]
oracle_stride:    [null]
oracle_tol:       ["1e-6"]
oracle_warmstart: [true]
path_length_norm: ["L2"]

# === Optimization / model stability ===
lambda_reg:             ["1e-4", "1e-3"]  # weak → strong convexity sweep
m_max:                  [1000]            # hard cap on deletes
d_max:                  [null]
lbfgs_pair_gate_m_t:    ["1e-3"]
lbfgs_spectrum_clip:    [["1e-6", "1e3"]] # [c_hat_min, C_hat_max]
ema_beta:               [0.9]
lambda_est_beta:        [0.1]
lambda_est_bounds:      [["1e-8", "1e4"]]

# === Calibration ===
bootstrap_iters:  [500]
quantile:         [0.95]

# === Dataset / generator (match your loaders) ===
dataset:          ["linear"]       # swap/extend: ["linear","covtype","cifar10","mnist"]
loss_name:        ["logistic"]     # or "squared" (must match what your runner expects)
